# Traveling Salesman Problem

José Luis Lobera del Castillo <br>
Rafael Andrade Ruíz Capetillo

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
import random

In [16]:
def read_csv_file(path):
    df = pd.read_csv(path, index_col=0)
    node_names = df.columns
    distance_matrix = df.values
    node_position_dict = dict(zip(node_names, list(range(len(node_names)))))

    return node_names, distance_matrix, node_position_dict


In [17]:
def GenerateNodes(n,lim):
    array_x= np.random.randint(lim, size=n)
    array_y= np.random.randint(lim, size=n)
    array_names=np.chararray(n)

    for i in range(0,n):
        array_names[i]=chr(65+i)

    #Para distancia exacta quitar que sea int 
    dist_matrix=np.zeros((n,n), dtype=int)
    for i in range(0,n):
        for j in range(0,n):
            if i!=j:
                dist_matrix[i,j]=math.sqrt((array_y[i]-array_y[j])**2+(array_x[i]-array_x[j])**2)
                
    #print(dist_matrix)
    dNodes={}
    for i in range(0,n):
        dNodes[array_names[i]]=i
    #print(dNodes)
    
    return array_names,dNodes,array_x,array_y,dist_matrix

In [18]:
# Use only with generated nodes

def PrintNodes(names,x,y):
    fig=plt.figure()
    ax=fig.add_axes([0,0,1,1])
    ax.scatter(x, y, color='r')

    for i in range(0,len(names)):
        ax.annotate(names[i], (x[i], y[i]))
        
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('NODES')
    
    plt.show()

In [19]:
def objectiveFunction(node_names, distance_matrix, individual, node_position_dict):
    individual = np.array(individual)

    if(individual[0] != individual[-1]): return -1                  # First and Last must be equal
    if(len(individual) != len(node_names)+1): return -1             # Shorter individual than expected
    if(len(node_names) != len(np.unique(individual))): return -1    # Unique values only
    #if(sorted(individual[:-1]) != sorted(node_names)): return -1

    distance = 0
    for i, node in enumerate(individual):
        if(i+1 >= len(individual)): break
    
        distance += distance_matrix[ node_position_dict[ individual[ i ]], node_position_dict[ individual[ i+1 ] ]]
        #print(i, node_position_dict[ individual[ i ]])
        #print(i, node_position_dict[ individual[ i+1 ]])
    return distance

In [7]:
node_names, distance_matrix, node_position_dict = read_csv_file('adjacency_matrix.csv')

## Genetic Algorithm

In [8]:
# Creates a permutation of all nodes
# Returns: List of the path,total distance

def GetRandomChromosome():
    individual = np.random.permutation(node_names[1:])
    individual = np.append(individual, node_names[0])
    individual = np.append(np.array([node_names[0]]), individual)
    distance = objectiveFunction(node_names, distance_matrix, individual, node_position_dict)
    return individual, distance

In [9]:
# Creates a population of random chromosomes
# Returns: List of Chromosomes

def GetPopulation(n = 0):
    return [GetRandomChromosome() for _ in range(n)] # O(n)

### Roulette Selection

In [10]:
# Sums the total weights of chromosomes
# Returns a list of proportions

def GetProportion(population):
    total = 0
    for chromosome in population:
        total += chromosome[1]
    
    proportion = []
    for chromosome in population:
        proportion.append(chromosome[1]/total)

    return proportion

In [11]:
def RouletteSelection(proportions):
    selection = random.random()

    acum = 0

    for i, prop  in enumerate(proportions):
        acum += prop
        if selection < acum:
            return i


##### Roulette Selected Chromosome

In [12]:
population = GetPopulation(10)

proportion = GetProportion(population)

# The way of getting a Roulette Selected Chromosome
selected = population[RouletteSelection(proportion)]
print(selected)

(array(['A', 'D', 'E', 'B', 'H', 'J', 'F', 'C', 'G', 'I', 'A'],
      dtype=object), 520)


### Tournament Selection

In [13]:
def TournamentSelection(population, k=2):
    community = random.sample(population, k)
    localElite = population[0]
    
    for chromosome in community:
        if chromosome[1] < localElite[1]:
            localElite = chromosome
            
    return localElite

##### Tournament Selected Chromosome

In [14]:
population = GetPopulation(10)

TournamentSelection(population, 5)

(array(['A', 'F', 'C', 'G', 'D', 'E', 'I', 'H', 'B', 'J', 'A'],
       dtype=object),
 383)